## Set up trial

In [19]:
from Reporter import LightConcurrentReporter
from MarkovDPAI import *
import numpy as np
import pandas as pd

def perform_trial(parameters):
    path_penalty = parameters.get('path_penalty')
    loss_penalty = parameters.get('loss_penalty')
    score_factor = parameters.get('score_factor')

    ai_input=ExpectiMax7(depth = 5, path_pen=path_penalty, loss_penalty=loss_penalty, score_factor=score_factor)
    r = LightConcurrentReporter(ai_input, 50)
    report = pd.DataFrame(r.generate_report(), columns=['score','max_tile'])
    print(np.mean(report['score']), len(report[report['max_tile']>=4096])/50.0)
    return np.mean(report['score'])
   # return len(report[report['max_tile']>=4096])/50.0

### Begin optimization

In [20]:
from ax.service.ax_client import AxClient
from ax.modelbridge.registry import Models
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep

generation_strategy = GenerationStrategy(
    steps=[
        GenerationStep(model=Models.SOBOL, num_trials=10),
        GenerationStep(model=Models.GPEI, num_trials=-1),   # GPEI for the rest of the trials
    ]
)
ax_client = AxClient(generation_strategy=generation_strategy)
# ax_client = AxClient()
ax_client.create_experiment(name='optimize Params',
                            parameters=[
                                {'name': 'path_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 10.0]},
                                {'name': 'loss_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 10.0]},
                                {'name': 'score_factor',
                                    'type': 'range',
                                    'bounds': [0.0, 5.0]},
                            ],
)

for i in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))


for i in range(110):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))

best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

[INFO 10-25 20:07:32] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 10-25 20:07:32] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter path_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-25 20:07:32] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter loss_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-25 20:07:32] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter score_factor. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[IN

3022.16 0.0


[INFO 10-25 20:08:23] ax.service.ax_client: Completed trial 1 with data: {'objective': (45635.2, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 10-25 20:08:23] ax.service.ax_client: Generated new trial 2 with parameters {'path_penalty': 6.818342, 'loss_penalty': 6.78875, 'score_factor': 2.227781} using model Sobol.


45635.2 0.5


[INFO 10-25 20:09:09] ax.service.ax_client: Completed trial 2 with data: {'objective': (48165.12, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 10-25 20:09:09] ax.service.ax_client: Generated new trial 3 with parameters {'path_penalty': 1.911929, 'loss_penalty': 3.219792, 'score_factor': 2.644202} using model Sobol.


...

37512.0 0.36


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 10-25 21:34:54] ax.service.ax_client: Generated new trial 118 with parameters {'path_penalty': 5.70931, 'loss_penalty': 10.0, 'score_factor': 2.046869} using model GPEI.
[INFO 10-25 21:35:36] ax.service.ax_client: Completed trial 118 with data: {'objective': (45689.52, None)}.


45689.52 0.5


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 10-25 21:35:38] ax.service.ax_client: Generated new trial 119 with parameters {'path_penalty': 9.656159, 'loss_penalty': 10.0, 'score_factor': 3.702807} using model GPEI.
[INFO 10-25 21:36:20] ax.service.ax_client: Completed trial 119 with data: {'objective': (43998.8, None)}.


43998.8 0.46
The best parameters are: {'path_penalty': 6.561649115294637, 'loss_penalty': 10.0, 'score_factor': 2.24397373076686}
with score ({'objective': 47303.358379203746}, {'objective': {'objective': 366674.2043493566}})


In [21]:
for i in range(120):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))
    
best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 10-26 09:02:45] ax.service.ax_client: Generated new trial 120 with parameters {'path_penalty': 6.500545, 'loss_penalty': 10.0, 'score_factor': 3.30883} using model GPEI.
[INFO 10-26 09:03:28] ax.service.ax_client: Completed trial 120 with data: {'objective': (44177.2, None)}.


44177.2 0.38


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 10-26 09:03:30] ax.service.ax_client: Generated new trial 121 with parameters {'path_penalty': 9.059648, 'loss_penalty': 0.0, 'score_factor': 3.537151} using model GPEI.
[INFO 10-26 09:04:21] ax.service.ax_client: Completed trial 121 with data: {'objective': (53560.48, None)}.


53560.48 0.54


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 10-26 09:04:23] ax.service.ax_client: Generated new trial 122 with parameters {'path_penalty': 8.848371, 'loss_penalty': 10.0, 'score_factor': 2.928657} using model GPEI.
[INFO 10-26 09:05:08] ax.service.ax_client: Completed trial 122 with data: {'objective': (45900.48, None)}.


...

45269.6 0.52


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 10-26 10:37:10] ax.service.ax_client: Generated new trial 238 with parameters {'path_penalty': 7.079687, 'loss_penalty': 8.031761, 'score_factor': 3.380305} using model GPEI.
[INFO 10-26 10:37:52] ax.service.ax_client: Completed trial 238 with data: {'objective': (41353.28, None)}.


41353.28 0.3


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
[INFO 10-26 10:37:59] ax.service.ax_client: Generated new trial 239 with parameters {'path_penalty': 10.0, 'loss_penalty': 10.0, 'score_factor': 3.294026} using model GPEI.
[INFO 10-26 10:38:43] ax.service.ax_client: Completed trial 239 with data: {'objective': (49443.12, None)}.


49443.12 0.52
The best parameters are: {'path_penalty': 9.723274060289766, 'loss_penalty': 0.0, 'score_factor': 3.5994970174523164}
with score ({'objective': 47110.32723688434}, {'objective': {'objective': 278119.7617427608}})


In [14]:
best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

data =  ax_client.experiment.fetch_data()
print(data.df)
print(data.df.sort_values(by='mean', ascending=False).head(10))

trial_params = ax_client.experiment.trials[132]
print(trial_params)

The best parameters are: {'path_penalty': 8.505974119306348, 'loss_penalty': 0.0, 'score_factor': 3.4897451874552705}
with score ({'objective': 0.8776559915826968}, {'objective': {'objective': 8.207984467722268e-05}})
    arm_name metric_name  mean  sem  trial_index
0        0_0   objective  0.84  NaN            0
1        1_0   objective  0.84  NaN            1
2        2_0   objective  0.08  NaN            2
3        3_0   objective  0.84  NaN            3
4        4_0   objective  0.84  NaN            4
..       ...         ...   ...  ...          ...
165    165_0   objective  0.86  NaN          165
166    166_0   objective  0.94  NaN          166
167    167_0   objective  0.90  NaN          167
168    168_0   objective  0.84  NaN          168
169    169_0   objective  0.86  NaN          169

[170 rows x 5 columns]
    arm_name metric_name  mean  sem  trial_index
166    166_0   objective  0.94  NaN          166
99      99_0   objective  0.94  NaN           99
72      72_0   objectiv

/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:199: RuntimeWarning: divide by zero encountered in divide
  return float(np.mean(1.96 * 2 * se_pred / np.abs(y_obs)))
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:229: RuntimeWarning: divide by zero encountered in scalar divide
  return float((np.max(y_obs) - min_y_obs) / min_y_obs)
